<a href="https://colab.research.google.com/github/yajima-yasutoshi/Model/blob/main/20250702/%E6%9C%80%E7%9F%AD%E8%B7%AF%E5%95%8F%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###準備
20250702


In [ ]:
%%capture
# Python MIPのインストール
!pip install mip

# 日本語環境のインストール
!pip install japanize-matplotlib

# 最短路問題

## 問題の概要

最短路問題（Shortest Path Problem）は、グラフ理論における基本的な問題の一つである。

今、
下の図のようなようなネットワーク（グラフ）が与えられ、
各枝（エッジ）には**重み**と呼ぶ実数が付与されているとする。
この時、指定された2点間（始点と終点）を結ぶ経路のうち、
最もコストの低い（距離が短い、時間が短いなど）経路を見つける問題が最短路問題である。
ここでの「コストの低い経路」とは、経路を構成する各枝に付随する重みの合計値を指す。


In [ ]:
#@title グラフ表示
import japanize_matplotlib # これをインポートすることで日本語表示が有効になる

# 必要なライブラリのインポート
import networkx as nx
import matplotlib.pyplot as plt
# グラフの作成
G = nx.Graph()

# ノードの追加 (A, B, C, D, E)
nodes = ["A", "B", "C", "D", "E"]
G.add_nodes_from(nodes)

# エッジと重みの追加 (ノード1, ノード2, 重み)
edges_with_weights = [
    ("A", "B", 4),
    ("A", "C", 2),
    ("B", "C", 5),
    ("B", "D", 10),
    ("C", "D", 3),
    ("C", "E", 7),
    ("D", "E", 4)
]
G.add_weighted_edges_from(edges_with_weights)

# グラフの描画設定
pos = nx.spring_layout(G, seed=42)  # ノードの配置を固定 (seedで再現性確保)
edge_labels = nx.get_edge_attributes(G, 'weight')

# 描画
plt.figure(figsize=(7, 4))
nx.draw_networkx_nodes(G, pos, node_size=700, node_color="skyblue")
nx.draw_networkx_edges(G, pos, width=2, alpha=0.6, edge_color="gray")
nx.draw_networkx_labels(G, pos, font_size=12) # font_family指定を削除 (japanize_matplotlibで自動設定)
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=16)

plt.title("簡単な重み付きグラフの例")
plt.axis("off") # 軸を非表示
plt.show()


グラフとは上の図のようなもので、A,B,C,D,Eを頂点（点、ノード、node）、
頂点と頂点を結ぶ線を枝（エッジまたはアーク、edge、arc）と呼ぶ。
特に、
各枝に重みが付与されている場合を、**重み付きグラフ（ネットワーク）**
と呼ぶことがある。


最短路問題は、現実世界の様々な場面で応用されている。例えば、カーナビゲーションシステムにおける最短経路探索、通信ネットワークにおけるデータパケットの最適ルーティング、物流における配送計画の最適化などが挙げられる。

## 定式化の詳細
最短路問題を定義するためには、重み付きグラフ（ネットワーク）を定める必要がある。
そこで、
重み付きグラフ（ネットワーク）の頂点の集合を$V$、
頂点を結ぶ枝の集合を$E$とし、
ノード $i$ からノード $j$ へ向かう枝$(i,j) \in E$には、
距離、時間、費用などの「コスト」$c_{ij}$ が割り当てられているとする。

### 変数の定義

この問題を数理モデルとして記述するために、以下のように変数を定義する。

各枝 $(i, j)\in E$ に対して、その枝を最短経路の一部として使用するかどうかを表すバイナリ変数（0または1の値を取る変数） $x_{ij}$ を定義する。

* $x_{ij} = 1$: 枝 $(i, j)$ を最短路の一部に使用する場合
* $x_{ij} = 0$: 枝 $(i, j)$ を最短路の一部に使用しない場合

### 目的関数

目的は、始点から終点までの経路のコスト最小化である。
各枝 $(i, j)$ のコストを $c_{ij}$ を使い、
目的関数は以下のように表される。

$$\min \sum_{(i,j) \in E} c_{ij} x_{ij}$$

ここで、$E$ はグラフの全ての枝の集合である。

### 制約条件

最短経路を構成するためには、変数 $x_{ij}$は
以下の制約条件を満たす必要がある。

1.  **フロー保存則（Flow Conservation Constraints）**:
    * **始点（Source Node, $s$）**: 始点からは1つの経路が出ていく。
    つまり、始点$s$から出ていく枝に対応する変数の合計は1である。
        $$\sum_{j: (s,j) \in E} x_{sj} = 1$$

    * **終点（Sink Node, $t$）**: 終点には1つの経路が流れ込む。
    つまり、終点$t$へ入ってくる枝に対応する変数の合計は1である。
        $$\sum_{i: (i,t) \in E} x_{it} = 1$$

  * **中間ノード（Intermediate Nodes, $k \neq s, t$）**: 始点でも終点でもない中間ノード $k$ では、入ってくる枝数の合計と、そのノードから出ていく枝数の合計は等しくなければならない。
$$\sum_{i: (i,k) \in E} x_{ik} = \sum_{j: (k,j) \in E} x_{kj} \quad \forall k \in V \setminus \{s, t\}$$
  ここで、$V$ はグラフの全てのノードの集合である。この式は以下のように書き換えることもできる。
$$\sum_{i: (i,k) \in E} x_{ik} - \sum_{j: (k,j) \in E} x_{kj} = 0 \quad \forall k \in V \setminus \{s, t\}$$

2.  **変数の定義域**:
各変数 $x_{ij}$ は0または1の値を取る。
$$x_{ij} \in \{0, 1\} \quad \forall (i,j) \in E$$
この条件により、この問題は整数計画問題（特に0-1整数計画問題）となる。もし枝の重みが全て非負であれば、この制約は $0 \le x_{ij} \le 1$ という連続変数として緩和しても、最適解においては自動的に $x_{ij}$ が0または1の値を取ることが知られており、その場合は線形計画問題として解くことができる。
本日の講義では、明示的にバイナリ変数として定義する。

## 数理モデル（数式表記）

上記の定式化をまとめると、最短路問題の数理モデルは以下のように表される。

**パラメータ:**

* $V$: ノードの集合
* $E$: 枝の集合
* $s \in V$: 始点ノード
* $t \in V$: 終点ノード
* $c_{ij}$: 枝 $(i, j) \in E$ のコスト

**変数:**

* $x_{ij} \in \{0, 1\}$: 枝 $(i, j) \in E$ を経路に含める場合に1、そうでない場合に0となる変数とする

**目的関数:**

$$\text{Minimize} \quad Z = \sum_{(i,j) \in E} c_{ij} x_{ij}$$

**制約条件:**

$$\sum_{j: (s,j) \in E} x_{sj} = 1 \quad \text{(始点)}$$
$$\sum_{i: (i,t) \in E} x_{it} = 1 \quad \text{(終点)}$$
$$\sum_{i: (i,k) \in E} x_{ik} - \sum_{j: (k,j) \in E} x_{kj} = 0 \quad \forall k \in V \setminus \{s, t\} \quad \text{(中間ノード)}$$
$$x_{ij} \in \{0, 1\} \quad \forall (i,j) \in E$$

## Python MIP を用いた実装例

ここでは、具体的なネットワークを例に取り、`python-mip` を用いて最短路問題を解く実装を示す。

### 例題ネットワーク
ノードと枝が以下のように定義された
ネットワークを考える。

* ノード: {0, 1, 2, 3, 4}
* 枝 (コスト):
    * (0, 1): 2
    * (0, 2): 4
    * (1, 2): 1
    * (1, 3): 5
    * (2, 3): 3
    * (2, 4): 1
    * (3, 4): 2

また、最短路は
ノード0を始点、ノード4を終点とする。


In [ ]:
#@title 可視化
import matplotlib.pyplot as plt
import networkx as nx

# ネットワークの定義
# (始点ノード, 終点ノード, コスト) のタプルで枝を表現
# 枝のリストを定義する
edges = [
    (0, 1, 2), (0, 2, 4),
    (1, 2, 1), (1, 3, 5),
    (2, 3, 3), (2, 4, 1),
    (3, 4, 2)
]

# ノードの集合を定める。
# ここでは集合型の変数でノード集合を表す。
# 集合型では.add() を使っても同じノードが2重に追加されない
nodes = set()
for u, v, c in edges:
    nodes.add(u)
    nodes.add(v)
num_nodes = len(nodes)

# 始点と終点
start_node = 0
end_node = 4

# グラフオブジェクトの作成
G = nx.DiGraph() # 有向グラフ

# ノードの追加
for n in nodes:
    G.add_node(n)

# 枝の追加とコストの属性設定
edge_labels = {}
for u, v, c in edges:
    G.add_edge(u, v, weight=c)
    edge_labels[(u,v)] = c

# ノードの配置を決定 (spring_layout, circular_layout, shell_layout など)
try:
  pos = nx.planar_layout(G)
except nx.NetworkXException:
  # グラフが平面的でない場合は、spring_layoutにフォールバックします
  print("グラフが平面的ではないため、spring_layoutを使用します。")
  pos = nx.spring_layout(G, seed=42) # 再現性のためにseedを設定

# 描画
plt.figure(figsize=(4, 4))
nx.draw(G, pos, with_labels=True, node_size=700, node_color="skyblue", font_size=14, font_weight="bold", arrowsize=20)
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red', font_size=14)

上のグラフで、ノード0からノード4に至る最短路を数理計画法で求めてみる。

##データの定義

In [ ]:
# ネットワークの定義
# (始点ノード, 終点ノード, コスト) のタプルで枝を表現
# 枝のリストを定義する
edges = [
    (0, 1, 2), (0, 2, 4),
    (1, 2, 1), (1, 3, 5),
    (2, 3, 3), (2, 4, 1),
    (3, 4, 2)
]

# ノードの集合を定める。
# ここでは集合型の変数でノード集合を表す。
# 集合型では.add() を使っても同じノードが2重に追加されない
nodes = set()
for u, v, c in edges:
    nodes.add(u)
    nodes.add(v)
num_nodes = len(nodes)

# 始点と終点
start_node = 0
end_node = 4


## python-mip による実装


In [ ]:
# mip から必要な関数や変数のみをインポートする
# mip.xsum のように mip. を付与する必要がない
from mip import Model, xsum, MINIMIZE, BINARY, OptimizationStatus

# モデルの作成
model = Model(name="shortest_path", sense=MINIMIZE )

各枝 $(i, j)\in E$ に対して、その枝を最短経路の一部として使用するかどうかを表すバイナリ変数（0または1の値を取る変数） $x_{ij}$ を定義する。

In [ ]:
# 変数の定義
# x[(u,v)] はノードuからノードvへの枝を使用するかどうかを表すバイナリ変数
x = {}
for u, v, c in edges:
    x[(u,v)] = model.add_var(name=f"x_{u}_{v}", var_type=BINARY)

### 目的関数

$$\min \sum_{(i,j) \in E} c_{ij} x_{ij}$$

ここで、$E$ はグラフの全ての枝の集合である。

In [ ]:
# 目的関数の設定
model.objective = xsum(c * x[(u,v)] for u, v, c in edges)

### 制約条件
  * **始点（Source Node, $s$）**: 始点からは1つの経路が出ていく。
    つまり、始点$s$から出ていく枝に対応する変数の合計は1である。
$$\sum_{j: (s,j) \in E} x_{sj} = 1$$

  * **終点（Sink Node, $t$）**: 終点には1つの経路が流れ込む。
    つまり、終点$t$へ入ってくる枝に対応する変数の合計は1である。
$$\sum_{i: (i,t) \in E} x_{it} = 1$$

  * **中間ノード（Intermediate Nodes, $k \neq s, t$）**: 始点でも終点でもない中間ノード $k$ では、入ってくる枝数の合計と、そのノードから出ていく枝数の合計は等しくなければならない。
$$\sum_{i: (i,k) \in E} x_{ik} - \sum_{j: (k,j) \in E} x_{kj} = 0 \quad \forall k \in V \setminus \{s, t\}$$

In [ ]:
# 制約条件の追加
for i in nodes:
    # ノードiから出ていく変数の和
    flow_out = xsum(x[(u,v)] for u, v, c in edges if u == i)
    # ノードiへ入ってくる変数の和
    flow_in = xsum(x[(u,v)] for u, v, c in edges if v == i)

    if i == start_node:
        model += (flow_out == 1, f"flow_conservation_start_{i}")
    elif i == end_node:
        model += (flow_in == 1, f"flow_conservation_end_{i}")
    else:
        model += (flow_out - flow_in == 0, f"flow_conservation_intermediate_{i}")

###最適化の実行と解の表示

In [ ]:
# モデルの求解
status = model.optimize()

# 結果の表示
if status == OptimizationStatus.OPTIMAL:
    print(f"最適解が見つかりました。")
    print(f"最短経路のコスト: {model.objective_value}")
    print("経路:")
    current_node = start_node
    path = [current_node]
    while current_node != end_node:
        for u, v, c in edges:
            if u == current_node and x[(u,v)].x >= 0.99: # ほぼ1であれば選択された枝
                print(f"  枝 ({u}, {v}) コスト: {c}")
                current_node = v
                path.append(current_node)
                break
    print(f"経路 (ノード列): {path}")
elif status == OptimizationStatus.INFEASIBLE:
    print("実行不可能: 問題に解が存在しません。")
elif status == OptimizationStatus.UNBOUNDED:
    print("非有界: 目的関数を無限に小さく（または大きく）できます。")
else:
    print(f"求解ステータス: {status}")

上記のコードを実行すると、始点0から終点4までの最短経路とその総コストが出力される。


### グラフ描画による可視化
最適解を可視化して確認する。

In [ ]:
#@title 可視化
import matplotlib.pyplot as plt
import networkx as nx

# グラフオブジェクトの作成
G = nx.DiGraph() # 有向グラフ

# ノードの追加
for n in nodes:
    G.add_node(n)

# 枝の追加とコストの属性設定
edge_labels = {}
for u, v, c in edges:
    G.add_edge(u, v, weight=c)
    edge_labels[(u,v)] = c

# ノードの配置を決定 (spring_layout, circular_layout, shell_layout など)
try:
  pos = nx.planar_layout(G)
except nx.NetworkXException:
  # グラフが平面的でない場合は、spring_layoutにフォールバックします
  print("グラフが平面的ではないため、spring_layoutを使用します。")
  pos = nx.spring_layout(G, seed=42) # 再現性のためにseedを設定

# 描画
plt.figure(figsize=(4, 4))
nx.draw(G, pos, with_labels=True, node_size=700, node_color="skyblue", font_size=14, font_weight="bold", arrowsize=20)
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red', font_size=14)

# 最適経路をハイライト (求解結果が得られている場合)
if status == OptimizationStatus.OPTIMAL:
    optimal_path_edges = []
    current_node = start_node
    while current_node != end_node:
        for u_edge, v_edge, c_edge in edges:
            if u_edge == current_node and x[(u_edge, v_edge)].x >= 0.99:
                optimal_path_edges.append((u_edge, v_edge))
                current_node = v_edge
                break
    nx.draw_networkx_edges(G, pos, edgelist=optimal_path_edges, width=2.5, edge_color="green", arrowsize=20)

plt.title("Network Graph and Shortest Path")
plt.show()

上の図では、求解された最短経路を緑色で示している。


## 実社会での応用例

最短路問題は、そのシンプルさにもかかわらず、非常に多くの実社会の問題に応用されている。

* **交通・運輸**:
    * **カーナビゲーション**: 出発地から目的地までの最短時間または最短距離の経路を探索する。
    * **公共交通機関の乗り換え案内**: 電車やバスの乗り換えを考慮して、目的地までの最短時間経路を提示する。
    * **航空路線の設計**: 航空会社がハブ空港を経由する効率的なフライトネットワークを構築する。
    * **配送計画**: 複数の配送先を巡回する際の最適な順路や、各地点への最短経路を決定する際に部分問題として利用される。
* **通信**:
    * **インターネットルーティング**: データパケットを送信元から宛先まで、ネットワークの混雑状況やリンクの帯域幅を考慮して、遅延が最小となるようにルーティングする（OSPFなどのプロトコルで利用）。
    * **電話網**: 通話の接続要求に対して、空いている回線を選んで最適な経路を確立する。
* **その他**:
    * **ロボットの経路計画**: 自律移動ロボットが障害物を避けながら目的地まで移動するための最短経路を計画する。
    * **画像処理**: 画像セグメンテーションなどで、特定の領域の境界線を見つける問題に応用されることがある。
    * **プロジェクト管理**: PERT (Program Evaluation and Review Technique) などで、プロジェクトのクリティカルパス（完了までに最も時間がかかる一連のタスク）を特定するのに使われる。これは最長路問題として定式化されるが、考え方は類似している。
    * **VLSI設計**: 半導体チップ上の配線経路を最適化する。

これらの応用例からもわかるように、最短路問題は様々な分野で意思決定を支援するための強力なツールとなっている。

## モデルの改善に関する簡単な視点

* **大規模問題への対応**: 非常に大規模なネットワーク（例えば、日本全国の道路網など）に対しては、この講義で示したような汎用的な整数計画ソルバーでは計算時間が非常に長くなる可能性がある。そのような場合には、ダイクストラ法のような、最短路問題に特化したより高速なアルゴリズムの利用が検討される。
ただし、これらのアルゴリズムは負のコストを持つ枝が存在する場合には適用できない（ベルマンフォード法は対応可能）などの制約がある。
* **動的なコスト**: 道路の渋滞状況のように、枝のコストが時間によって変化する場合、時間依存最短路問題 (Time-Dependent Shortest Path Problem) となり、より複雑なモデル化やアルゴリズムが必要となる。
* **複数目的**: 単に距離だけでなく、時間や料金など複数の要素を同時に最適化したい場合、多目的最適化問題として捉える必要がある。
* **制約の追加**: 特定のノードを経由する、あるいは経由しないといった追加の制約がある場合、それらをモデルに組み込むことで対応できる。例えば、「ノード $k$ を必ず経由する」という制約は、始点から $k$ までの最短路と、$k$ から終点までの最短路をそれぞれ求め、それらを結合することで実現できる。

数理計画モデリングの利点は、これらの複雑な要件を制約条件として柔軟に追加できる点にある。


## 主要なアルゴリズムの紹介 (数理計画法以外の専用解法)

特に大規模なグラフに対しては、専用に設計されたより高速なアルゴリズムが実用上広く用いられています。
これらのアルゴリズムの詳細はこの講義では扱わないが、各自で調べて学習することを
強く推奨する。

  * **ダイクストラ法 (Dijkstra's Algorithm)**:

      * **概要**: 単一始点・全終点最短路問題を解くための代表的なアルゴリズム。始点から未確定のノードへの最短距離を段階的に確定させていく方法で、
      ラベリング法とも呼ばれる。
      * **動作原理**:
        1.  始点ノードの距離を0、他の全ノードの距離を無限大に初期化。
        2.  未確定ノードの中で最も距離が小さいノードを選び、そのノードを確定済みにする。
        3.  確定したノードに隣接する未確定ノードについて、そのノードを経由した場合の新しい距離を計算し、もし現在の距離より短ければ更新する。
        4.  全てのノードが確定するまで2-3を繰り返す。
      * **条件**: 全ての辺の**重みが非負**である場合にのみ正しく動作する。
      * **`networkx`パッケージでの利用が可能**: `nx.dijkstra_path(G, source, target, weight='weight')` や `nx.single_source_dijkstra_path_length(G, source, weight='weight')` などで簡単に利用できる。

  * **ベルマンフォード法 (Bellman-Ford Algorithm)**:
    * **概要**: こちらも単一始点・全終点最短路問題を解くアルゴリズム。ダイクストラ法と異なり、**負の重みを持つ辺があっても動作する**。
    * **負の閉路の検出**: $|V|$ 回目の緩和操作でも距離が更新される場合、グラフ内に始点から到達可能な負の閉路が存在することを示すことができる（負の閉路があると最短路が定義できない）。
    * **計算量**: ダイクストラ法より遅いが、適用範囲が広い。
    * **`networkx`パッケージでの利用**: `nx.bellman_ford_path(G, source, target, weight='weight')` など。

  * **ワーシャル・フロイド法 (Warshall-Floyd Algorithm)**:
      * **概要**: 全点対最短路問題を解くアルゴリズム。動的計画法に基づき、全ての中間ノードを考慮して全ペア間の最短路を更新していく。

MIPによる解法は非常に柔軟で様々な制約を加えやすい利点がありますが、純粋な最短路問題（特に大規模なもの）に対しては、これらの専用アルゴリズムの方が計算効率の面で優れています。

---
#演習問題

## 演習問題1:

以下のネットワークについて、ノードSからノードGへの最短経路のコストを、`python-mip` を用いて求めよ。

* ノード: {S, A, B, C, D, G}
* 枝 (コスト):
    * (S, A, 1)
    * (S, B, 4)
    * (A, B, 2)
    * (A, C, 1)
    * (A, D, 3)
    * (B, D, 1)
    * (C, G, 2)
    * (D, C, 1)
    * (D, G, 6)
* 始点: S
* 終点: G


##演習問題2:

以下の都市間の道路網を考える。道路には一方通行のものと両方通行のものがあり、それぞれ異なるコスト（所要時間）がかかる。
ノード1からノード5への最短コスト（所要時間）を求めよ。

* ノード: {1, 2, 3, 4, 5}
* 枝 (始点, 終点, コスト):
    * (1, 2, 10)  両方向
    * (1, 3, 15)  両方向
    * (2, 3, 5)   2から3への一方通行
    * (2, 4, 20)  両方向
    * (3, 4, 5)   両方向
    * (3, 5, 25)  3から5への一方通行
    * (4, 5, 10)  両方向
* 始点: 1
* 終点: 5

**考え方**
例えば枝 (1, 2, 10) が両方向に移動が可能であれば、
枝として (1, 2, 10) と (2, 1, 10) を定義する。



##演習問題3:

演習問題1のネットワークにおいて、ノードSからノードGへ行く際に、**必ずノードDを経由する**という条件下での最短経路のコストを求めよ。

**考え方:**
この問題は2つの独立した最短路問題として解くことができる。
1.  SからDへの最短路
2.  DからGへの最短路
そして、これらの結果を組み合わせる。それぞれの最短路問題は `python-mip` で個別にモデル化して解く。


## 演習問題4:

演習問題1のネットワークにおいて、枝 (A, C) のコストが1から10に増加した場合、ノードSからノードGへの最短経路とそのコストはどのように変化するか。変更後のネットワークで最短コストを求めよ。


##演習問題5:

演習問題1のネットワークにおいて、各枝には通常のコスト（時間）に加えて、「特別料金」が設定されているとする。SからGへの最短時間経路を見つけたいが、支払う「特別料金」の合計が**4以内**でなければならない。
この条件下での最短コスト（時間）を求めよ。

* ノード: {S, A, B, C, D, G}
* 枝 (始点, 終点, 時間コスト, 特別料金):
    * (S, A, 1, 1)
    * (S, B, 4, 0)
    * (A, B, 2, 1)
    * (A, C, 5, 2)
    * (A, D, 3, 3)
    * (B, D, 1, 3)
    * (C, G, 2, 1)
    * (D, C, 1, 0)
    * (D, G, 6, 2)
* 始点: S
* 終点: G
* 特別料金上限: 4

**考え方:**
目的関数は時間の最小化のままである。追加の制約条件として、選択された枝の特別料金の合計が上限を超えないようにする。


## 演習問題6:

ある配送員が始点Xから出発し、目的地Aと目的地Bの両方に荷物を届ける必要がある。
総移動コストの最小値を解答せよ。
ただし、荷物を届けた後、始点Xに戻る必要はない。また、AとBのどちらを先に訪れても良い。

以下のネットワークを使用する。
* ノード: {X, A, B, C, D}
* 枝 (コスト):
    * (X, A, 5), (X, C, 2)
    * (A, X, 5), (A, B, 3), (A, D, 6)
    * (B, A, 3), (B, D, 2)
    * (C, X, 2), (C, A, 4), (C, D, 7)
    * (D, A, 6), (D, B, 2), (D, C, 7)
* 始点: X
* 目的地: A, B

**考え方:**
1.  以下の3つの区間の最短路をそれぞれ求める必要がある。
    * X から A
    * X から B
    * A から B (両方向のコストが同じなら B から A も同じ)
2.  2つの訪問順序（X→A→B と X→B→A）の総コストを計算し比較する。
